Loading the data

id: household

Choices:  
- heinz41  
- heinz32  
- heinz28  
- hunts32  

variables  
- disp (1/0)  
- feat (1/0)  
- price (Log?)  

In [14]:
import numpy as np
import pandas as pd
df = pd.read_pickle("data.pkl") 
df.head()

,id,disp.heinz41,disp.heinz32,disp.heinz28,disp.hunts32,feat.heinz41,feat.heinz32,feat.heinz28,feat.hunts32,price.heinz41,price.heinz32,price.heinz28,price.hunts32,choice,choiceindex
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz28,2
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz28,2
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz28,2


Defining the likelihood function using numba and numpy arrays and multiple draws

In [15]:
import numpy as np
import pandas as pd
import datetime
import time
from scipy.optimize import minimize
from numba import jit, prange

@jit(nopython=True, parallel=True)
def likelihood(c, data, draws, verbose=False):
    #print("evaluation likelihood")
    n_r = draws.shape[0]
    n_q = draws.shape[1]
#   n_k = draws.shape[2]
    
    #matrix with all simulations
    simulations = np.zeros((n_q, n_r))
    
    #iterate over households
    for q in prange(n_q):
        rows = data[np.where(data[:,0] == q+1)]
        n_rows = len(rows)
        
        #iterate over draws per household
        for r in prange(n_r):
            probabilities= np.zeros(n_rows)
            
            #iterate over oberservations per househould
            for t in prange(n_rows):
                choices = np.zeros(4)
                
                #itetate over probability of choices per observation
                for j in prange(4):
                    utility = 0; #start with alpha
                    if j < 3: utility = c[j]
                    x = [rows[t][1+j], rows[t][5+j], rows[t][9+j]]
                    mu =  c[3:6]
                    sigma = c[6:]
                    #np.exp(alpha +  np.dot(c[3:6],x) + np.dot(np.multiply(c[6:],draws[row[0]-1]),x))
                    for l in prange(3):
                          utility += mu[l] * x[l] + np.absolute(sigma[l]) * draws[r][q][l] * x[l]

                    choices[j] = np.exp(utility)
            
                probabilities[t] = choices[int(rows[t][13])] / np.sum(choices)
                
            simulations[q,r] = np.exp(np.log(probabilities).sum())
            
    estimates = np.zeros(n_q)
    for q in prange(n_q):
        estimates[q] = np.sum(simulations[q,:]) / n_r #.mean()
    
    res = -np.log(estimates).sum()
    
    if verbose == 2: print(res)
        
    return res

iterations = 1

   
def mixedlogit(data, drawtype, n_draws, c_0=False, method='BFGS', verbose=0):
    n_q = len(data.id.unique())
    
#     coefficients = [#alpha heinz41
#                     #alpha heinz32 
#                     #alpha heinz28
#                     #mu    display
#                     #mu    feat
#                     #mu    price
#                     #sigma dispay 
#                     #sigma feat 
#                     #sigma price 
#                     ]
    
    #generate inital values if neccesary
    if isinstance(c_0, bool) and c_0 == False:
            #genereate random starting coefficients
            c_0 = np.random.rand(9)
    elif len(c_0) != 9:
        raise Exception("Incorrect initial coefficients")
    
    #generate draws
    if drawtype == 'pseudo':
        draws = np.random.randn(n_draws, n_q, 3); 
    else:
         raise Exception("Incorrect Drawtype")
    
    
    global iterations
    iterations = 1
    def logging(xk):
        global iterations
        if(verbose >0):
            print("Iteration %d, coefficients:" % (iterations))
            print(xk)
            print('\n')
            iterations += 1
    
    start = time.time()
    res =  minimize(likelihood, c_0, args=(data.drop(columns='choice').values, draws, verbose), method=method, callback=logging)
    end = time.time()
    duration = end-start 
    
    #if verbose: 
    print("Optimization done, time elapsed: %s" % str(datetime.timedelta(seconds=round(duration))))
    
    res['duration'] = duration
    return res

benchmarking and comparing pandas and numba implementation

In [ ]:
np.random.seed(1234)

#example run with BFGS
print(mixedlogit(df, drawtype="pseudo", n_draws=100, method='BFGS', verbose=False))

#example run with Nelder-Mead
print(mixedlogit(df, drawtype="pseudo", n_draws=100, method='Nelder-Mead', verbose=False))

In [16]:
import pickle
example_dict = pickle.load(open("datasets/n 10 - seed 1234 - time 2019-06-17-193522.pickle","rb"))
example_dict['data'][0]

,id,disp.heinz41,disp.heinz32,disp.heinz28,disp.hunts32,feat.heinz41,feat.heinz32,feat.heinz28,feat.hunts32,price.heinz41,price.heinz32,price.heinz28,price.hunts32,choice,choiceindex
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz32,1
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz32,1
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz32,1
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz32,1
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz32,1
5,1,0,0,0,0,0,0,0,0,5.0,3.0,4.7,3.0,heinz32,1
6,1,0,0,0,1,0,0,0,1,5.1,3.1,4.6,4.1,heinz32,1
7,1,0,0,0,0,0,0,0,0,4.6,3.4,4.7,3.1,heinz32,1
8,1,0,0,0,0,0,0,0,0,5.0,3.4,4.7,3.1,heinz32,1
9,1,0,0,0,1,0,0,0,0,5.0,3.4,5.0,2.8,hunts32,3


In [26]:
np.random.seed(1234)

c = [1.8, #alpha heinz41
     1.2, #alpha heinz32 
     2.0, #alpha heinz28
     1.3, #mu    display
     1.1, #mu    feat
     -1.8, #mu    price
     0.25, #sigma dispay 
     0.5, #sigma feat 
     1.2 #sigma price 
    ]

c = np.multiply(c,1)

regression = mixedlogit(example_dict['data'][1], c_0=c, drawtype="pseudo", n_draws=100, method='Nelder-Mead', verbose=1)
print(regression)

Iteration 1, coefficients:
[ 1.62        1.22        2.03333333  1.32166667  1.11833333 -1.83
  0.25416667  0.50833333  1.22      ]


Iteration 2, coefficients:
[ 1.62        1.22        2.03333333  1.32166667  1.11833333 -1.83
  0.25416667  0.50833333  1.22      ]


Iteration 3, coefficients:
[ 1.62        1.22        2.03333333  1.32166667  1.11833333 -1.83
  0.25416667  0.50833333  1.22      ]


Iteration 4, coefficients:
[ 1.62        1.22        2.03333333  1.32166667  1.11833333 -1.83
  0.25416667  0.50833333  1.22      ]


Iteration 5, coefficients:
[ 1.62        1.22        2.03333333  1.32166667  1.11833333 -1.83
  0.25416667  0.50833333  1.22      ]


Iteration 6, coefficients:
[ 1.66610882  1.25950719  2.09917865  1.36446612  1.02621492 -1.88926078
  0.2485656   0.50216092  1.21506274]


Iteration 7, coefficients:
[ 1.66610882  1.25950719  2.09917865  1.36446612  1.02621492 -1.88926078
  0.2485656   0.50216092  1.21506274]


Iteration 8, coefficients:
[ 1.66610882  1.2595071

In [22]:
res = [ 44.52974914,  28.01881169,  50.01573162,  30.17899965,
        33.54040419, -40.38707385,  -6.6163476 , -18.71561041,
        30.09048101]

c = [1.8, #alpha heinz41
     1.2, #alpha heinz32 
     2.0, #alpha heinz28
     1.3, #mu    display
     1.1, #mu    feat
     -1.8, #mu    price
     0.25, #sigma dispay 
     0.5, #sigma feat 
     1.2 #sigma price 
    ]

np.multiply(res, 0.04042241500936513)

array([ 1.8       ,  1.13258803,  2.02175666,  1.21990805,  1.35578414,
       -1.63254306, -0.26744875, -0.75653017,  1.21632991])

In [28]:
regression.x
np.multiply(, 1.8/res2[0])

array([ 3.30533178e+01,  2.22922909e+01,  3.71011209e+01,  5.65872545e+01,
        2.06715858e+01, -2.71822142e+01, -4.03425271e+01,  5.30769384e-03,
        2.35815031e+01])